In [32]:
import os
from dotenv import load_dotenv
from google.cloud import storage, bigquery
from google.oauth2 import service_account
from datetime import datetime, timedelta, timezone
import pandas as pd
import requests

In [37]:
load_dotenv()
info = {
  "project_id": os.getenv("GCP_PROJECT_ID"),
  "client_email": os.getenv("GCP_CLIENT_EMAIL"),
  "private_key": os.getenv("GCP_PRIVATE_KEY").replace("\\n","\n"),
  "token_uri": "https://oauth2.googleapis.com/token"
}
credential = service_account.Credentials.from_service_account_info(info)
storage_client = storage.Client(credentials=credential)
bigquery_client = bigquery.Client(credentials=credential)
# query from bigquery
query = """
  SELECT delegate,sum(cast(amount as numeric)) as voting_power FROM `curia-dao.curia_op_indexer.holders` where delegate is not null group by delegate order by sum(cast(amount as numeric)) desc limit 100
"""
query_job = bigquery_client.query(query)
delegates = []
rank = 1
current = datetime.now(timezone.utc)
for row in query_job:
  delegates.append({
    'rank': rank,
    'delegate': row[0],
    'amount': str(row[1]),
    'date': (current - timedelta(days=1)).strftime("%Y-%m-%d"),
    'fetch_timestamp': current.strftime("%Y-%m-%d %H:%M:%S %Z%z")
  })
  rank += 1
# write json file
df = pd.DataFrame(delegates)
df.to_json('delegates_without_partial_vp.json', orient='records')
bucket = storage_client.bucket('dynamic_attestation_public')
blob = bucket.blob(f'mvp/{current.strftime("%Y-%m-%d")}/delegates_without_partial_vp.json')
blob.upload_from_filename('delegates_without_partial_vp.json')
blob.make_public()
os.remove('delegates_without_partial_vp.json')
# call curia api
params = {
  'limit': 100,
  'page': 1,
  'sort': 'delegateToken',
  'isAsc': False
}
url = 'https://prod.op.api.curiahub.xyz/api/delegates'
results = requests.get(url, params=params).json()
all_delegates = []
for delegate in results['delegates']:
  all_delegates.append({
    'rank': delegate['delegateRank'],
    'delegate': delegate['delegateAddress'],
    'ens_address': delegate['ensAddress'],
    'voting_power': delegate['delegateToken'],
    'vp': delegate['legacyDelegate'],
    'partial_vp': delegate['partialDelegate'],
    'date': (current - timedelta(days=1)).strftime("%Y-%m-%d"),
    'fetch_timestamp': current.strftime("%Y-%m-%d %H:%M:%S %Z%z")
  })
# write json file
df = pd.DataFrame(all_delegates)
df.to_json('delegates_with_partial_vp.json', orient='records')
blob = bucket.blob(f'mvp/{current.strftime("%Y-%m-%d")}/delegates_with_partial_vp.json')
blob.upload_from_filename('delegates_with_partial_vp.json')
blob.make_public()
os.remove('delegates_with_partial_vp.json')
